In [19]:
import dash
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [20]:
df = pd.read_csv('https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/ECARO,TRANSMONEE,1.0/.EDUNF_STU_L2_TOT+EDUNF_STU_L2_TOT+EDUNF_STU_L3_TOT....?format=csv')

In [21]:
df['Geographic area'] 

,Geographic area,Indicator,Sex,Age,Residence,Wealth Quintile,TIME_PERIOD,OBS_VALUE,Unit of measure,OBS_FOOTNOTE,Frequency,DATA_SOURCE,Unit multiplier,OBS_STATUS
0,Albania,Lower Secondary education (ISCED 2) enrolments...,Total,Total,Total,Total,1998,260160.0,Persons,NaN,Annual,UIS,Units,A
1,Albania,Lower Secondary education (ISCED 2) enrolments...,Total,Total,Total,Total,1999,261341.0,Persons,NaN,Annual,UIS,Units,A
2,Albania,Lower Secondary education (ISCED 2) enrolments...,Total,Total,Total,Total,2000,260718.0,Persons,NaN,Annual,UIS,Units,A
3,Albania,Lower Secondary education (ISCED 2) enrolments...,Total,Total,Total,Total,2001,266940.0,Persons,NaN,Annual,UIS,Units,A
4,Albania,Lower Secondary education (ISCED 2) enrolments...,Total,Total,Total,Total,2002,259650.0,Persons,NaN,Annual,UIS,Units,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3336,Uzbekistan,Upper secondary education (ISCED 3) enrolment ...,Male,Total,Total,Total,2013,869167.0,Persons,NaN,Annual,UIS,Units,A
3337,Uzbekistan,Upper secondary education (ISCED 3) enrolment ...,Male,Total,Total,Total,2014,834877.0,Persons,NaN,Annual,UIS,Units,A
3338,Uzbekistan,Upper secondary education (ISCED 3) enrolment ...,Male,Total,Total,Total,2015,800425.0,Persons,NaN,Annual,UIS,Units,A
3339,Uzbekistan,Upper secondary education (ISCED 3) enrolment ...,Male,Total,Total,Total,2016,768232.0,Persons,NaN,Annual,UIS,Units,A


In [22]:
# Build AppViewer 
from jupyterlab_dash import AppViewer
viewer = AppViewer()

available_indicators = df['Indicator'].unique()


In [29]:


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Enrollments'
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Enrollment Level'
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Albania'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        id='crossfilter-year--slider',
        min=df['TIME_PERIOD'].min(),
        max=df['TIME_PERIOD'].max(),
        value=df['TIME_PERIOD'].max(),
        marks={str(year): str(year) for year in df['TIME_PERIOD'].unique()},
        step=None
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['TIME_PERIOD'] == year_value]

    return {
        'data': [dict(
            x=dff[dff['Indicator'] == xaxis_column_name]['OBS_VALUE'],
            y=dff[dff['Indicator'] == yaxis_column_name]['OBS_VALUE'],
            text=dff[dff['Indicator'] == yaxis_column_name]['Geographic area'],
            customdata=dff[dff['Indicator'] == yaxis_column_name]['Geographic area'],
            mode='markers',
            marker={
                'size': 25,
                'opacity': 0.7,
                'color': 'orange',
                'line': {'width': 2, 'color': 'purple'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }


def create_time_series(dff, axis_type, title):
    return {
        'data': [dict(
            x=dff['TIME_PERIOD'],
            y=dff['OBS_VALUE'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }


@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Geographic area'] == country_name]
    dff = dff[dff['Indicator'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Geographic area'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

In [30]:
app.run_server(mode="jupyterlab")
